By Raiyan Abdul Baten

University of Rochester, NY, USA

Mail: raiyanabdulbaten@gmail.com

# K-NN regression 
## Implementation and Cheat Sheet

This page documents the summary theory and implementation of K-NN regression.

In [1]:
import pandas as pd
import os
import numpy as np
import re
import csv
from IPython.display import display, Math, Latex
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
%matplotlib inline

def load_csv_data(folder_name, file_name, dtype_dict=None):
    csv_path = os.path.join(folder_name, file_name+".csv")
    return pd.read_csv(csv_path, dtype=dtype_dict)

## Loading data

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, \
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, \
              'price':float, 'bedrooms':float, 'zipcode':str, \
              'long':float, 'sqft_lot15':float, 'sqft_living':float, \
              'floors':str, 'condition':int, 'lat':float, 'date':str, \
              'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
def dataframe_prepare(dataframe,features,target):
    dataframe["constant"]=1
    one_padded_features=["constant"]
    one_padded_features.extend(features)
    X = dataframe[one_padded_features].values
    Y = dataframe[target].values
    return X,Y

In [4]:
df_features_train = load_csv_data("data","kc_house_data_small_train",dtype_dict)
df_features_test = load_csv_data("data","kc_house_data_small_test",dtype_dict)
df_features_valid = load_csv_data("data","kc_house_data_small_validation",dtype_dict)

simple_features = ['bedrooms','bathrooms','sqft_living','sqft_lot','waterfront',\
                   'view','condition','grade','sqft_above','sqft_basement','yr_built',\
                   'yr_renovated','lat','long','sqft_living15','sqft_lot15']
X_train,Y_train=dataframe_prepare(df_features_train,simple_features,['price'])
X_test,Y_test=dataframe_prepare(df_features_test,simple_features,['price'])
X_valid,Y_valid=dataframe_prepare(df_features_valid,simple_features,['price'])

## Normalizing features

In [5]:
def normalize_features(X):
    norms = np.linalg.norm(X, axis=0)
    X_normalized = X / norms
    return (X_normalized, norms)

In [6]:
X_train_normalized, norms = normalize_features(X_train)
X_test_normalized = X_test / norms
X_valid_normalized = X_valid / norms

## K-NN Algorithm

In [7]:
def compute_distances(features_instances, features_query):
    diff=features_instances - features_query
    distances = np.sqrt(np.sum((diff)**2,axis=1))
    return distances

In [8]:
def k_nearest_neighbors(k, feature_train, features_query,print_steps):
    distances = compute_distances(feature_train, features_query)
    if(print_steps):
        print("distances: "+str(distances))
    idx = np.argsort(distances) #idx[0] is smallest
    if(print_steps):
        print("sorted indices: "+str(idx))
    neighbors = idx[0:k]
    if(print_steps):
        print("selected indices: "+str(neighbors))
    return neighbors

In [9]:
def predict_output_of_query(k, features_train, output_train, query_set, print_steps=False):
    prediction_list=[]
    for i in range(query_set.shape[0]):
        if(print_steps):
            print("query set: "+str(query_set[i,:]))
        neighbors= k_nearest_neighbors(k, features_train, query_set[i,:],print_steps)
        selected_outputs = output_train[neighbors]
        if(print_steps):
            print("selected outputs: "+str(selected_outputs))        
        prediction = np.mean(selected_outputs)
        if(print_steps):
            print("Predicted value: "+str(prediction))
        prediction_list.append(prediction)
        if(print_steps):
            print(" ")
    return prediction_list

### Testing the algorithm on a toy dataset
XX has 4 2D datapoints, each corresponding to a target value in YY. Given a query_set of new datapoints, the $k=2$ nearest datapoints are selected, and the average of their target values are reported as the regression results for each of the query datapoints.

In [10]:
XX= np.array([[1,2],[1,3],[4,5],[9,10]])
print(XX)

[[ 1  2]
 [ 1  3]
 [ 4  5]
 [ 9 10]]


In [11]:
YY = np.array([[2],[3],[4],[9]])
print(YY)

[[2]
 [3]
 [4]
 [9]]


In [12]:
query_set = np.array([[1,2],[8.7,8.8]])
prediction_list=predict_output_of_query(2, XX, YY, query_set, print_steps=True)

query set: [1. 2.]
distances: [ 0.          1.          4.24264069 11.3137085 ]
sorted indices: [0 1 2 3]
selected indices: [0 1]
selected outputs: [[2]
 [3]]
Predicted value: 2.5
 
query set: [8.7 8.8]
distances: [10.27277957  9.64002075  6.04400529  1.23693169]
sorted indices: [3 2 1 0]
selected indices: [3 2]
selected outputs: [[9]
 [4]]
Predicted value: 6.5
 


### Testing the algorithm on a large dataset
Using the dataset preprocessed above.

In [13]:
prediction_list=predict_output_of_query(10, X_train_normalized, Y_train, X_test_normalized,print_steps=True)

query set: [ 0.01345102  0.01551285  0.01807473  0.01759212  0.00160518  0.
  0.05102365  0.0116321   0.01564352  0.01362084  0.02481682  0.01350306
  0.          0.01345387 -0.01346922  0.01375926  0.0016225 ]
distances: [0.05966815 0.08546881 0.0609051  ... 0.05716729 0.05705739 0.05784207]
sorted indices: [3758 3144 4208 ... 1107 5226 2486]
selected indices: [3758 3144 4208 5038 4614 4695 1495  507 1648 4039]
selected outputs: [[ 635000.]
 [1340000.]
 [1260000.]
 [ 460000.]
 [ 612000.]
 [1230000.]
 [ 695000.]
 [ 345000.]
 [ 545000.]
 [ 960000.]]
Predicted value: 808200.0
 
query set: [ 0.01345102  0.01551285  0.00602491  0.00954149  0.00138046  0.
  0.          0.01550947  0.01216718  0.01100674  0.          0.0130732
  0.          0.01348029 -0.0134657   0.01035159  0.00174419]
distances: [0.00694764 0.06831362 0.01385434 ... 0.01307039 0.01464324 0.01087014]
sorted indices: [4670 1385 4834 ... 1107 5226 2486]
selected indices: [4670 1385 4834 4033 2226  887 4700 4679 1127 1847]
se

selected indices: [ 616 5044 2804 5369 5428 4418 5245 5095 1599 2523]
selected outputs: [[449950.]
 [505657.]
 [540000.]
 [395000.]
 [499990.]
 [725000.]
 [572000.]
 [515000.]
 [367500.]
 [784950.]]
Predicted value: 535504.7
 
query set: [ 0.01345102  0.01163464  0.00602491  0.0059038   0.00238209  0.
  0.          0.01938684  0.01042901  0.00681042  0.          0.01337342
  0.          0.01337423 -0.01345227  0.00649386  0.00300974]
distances: [0.00845545 0.06943893 0.01417293 ... 0.01887467 0.02174831 0.01411357]
sorted indices: [4102  169 4406 ... 1107 5226 2486]
selected indices: [4102  169 4406 4518 2173 1055 3504  784  836 1255]
selected outputs: [[205000.]
 [218000.]
 [189950.]
 [217000.]
 [189000.]
 [291500.]
 [191000.]
 [203000.]
 [206000.]
 [245000.]]
Predicted value: 215545.0
 
query set: [ 0.01345102  0.01551285  0.0165685   0.01395443  0.00203858  0.
  0.          0.01550947  0.01216718  0.00901177  0.02635188  0.01346895
  0.          0.01344949 -0.01346966  0.01298771  0

sorted indices: [4489 3347 3842 ... 1107 5226 2486]
selected indices: [4489 3347 3842 4468 1596 3578  535  442 2521 4402]
selected outputs: [[605004.]
 [492000.]
 [370000.]
 [447500.]
 [670000.]
 [397950.]
 [550000.]
 [561000.]
 [487000.]
 [485000.]]
Predicted value: 506545.4
 
query set: [ 0.01345102  0.00775643  0.00903736  0.00882588  0.00043308  0.
  0.          0.0116321   0.01216718  0.00853022  0.00614024  0.01369411
  0.          0.0134461  -0.01347043  0.00996582  0.00054719]
distances: [0.00849049 0.06760022 0.01225377 ... 0.01381536 0.01752123 0.00980615]
sorted indices: [5205 5105 5366 ... 1107 5226 2486]
selected indices: [5205 5105 5366 5055 1885 3657 5043 5464 4228 3068]
selected outputs: [[390000.]
 [342500.]
 [450000.]
 [379900.]
 [441000.]
 [550000.]
 [349000.]
 [403000.]
 [313500.]
 [370000.]]
Predicted value: 398890.0
 
query set: [ 0.01345102  0.01163464  0.00602491  0.00542672  0.00277953  0.
  0.          0.0116321   0.01216718  0.00626008  0.          0.01333931

selected indices: [5135 4149 1644  328 1654 1426 1759 1621 1748 3489]
selected outputs: [[1238000.]
 [1387000.]
 [ 817500.]
 [1875000.]
 [1925000.]
 [2400000.]
 [1488000.]
 [1436000.]
 [ 625000.]
 [1712500.]]
Predicted value: 1490400.0
 
query set: [ 0.01345102  0.01551285  0.01506227  0.01961969  0.01123659  0.
  0.          0.0116321   0.01738169  0.0226326   0.          0.01358494
  0.          0.01349678 -0.01343213  0.0262969   0.01637549]
distances: [0.03305683 0.07172347 0.03210066 ... 0.02317774 0.0208506  0.03035669]
sorted indices: [3058 1260 2763 ... 1107 5226 2486]
selected indices: [3058 1260 2763 1192 3184  414 2317  285 3839 3435]
selected outputs: [[ 940000.]
 [ 895000.]
 [ 950000.]
 [ 925000.]
 [ 845000.]
 [ 650000.]
 [ 685000.]
 [1050000.]
 [ 690000.]
 [ 896000.]]
Predicted value: 852600.0
 
query set: [ 0.01345102  0.01163464  0.01204982  0.01210577  0.00797101  0.
  0.          0.01550947  0.01216718  0.00839264  0.02072332  0.01350306
  0.          0.01336914 -0.01

distances: [0.01829587 0.0679301  0.02084887 ... 0.02085526 0.02285327 0.01852662]
sorted indices: [ 342 3411 5021 ... 1107 5226 2486]
selected indices: [ 342 3411 5021 3354 4065  199 2662 2305  337  495]
selected outputs: [[532500.]
 [520000.]
 [663000.]
 [641000.]
 [215000.]
 [320000.]
 [655000.]
 [535000.]
 [317000.]
 [375000.]]
Predicted value: 477350.0
 
query set: [ 0.01345102  0.00775643  0.01054359  0.00751392  0.00170149  0.
  0.          0.01550947  0.01216718  0.00577854  0.01074543  0.01331201
  0.          0.01348484 -0.01346625  0.00822984  0.00121688]
distances: [0.01315201 0.06795862 0.01596676 ... 0.01854474 0.02187457 0.0141067 ]
sorted indices: [ 853 1143 2048 ... 1107 5226 2486]
selected indices: [ 853 1143 2048  872  319 1849  609 1332 3915 3419]
selected outputs: [[390000.]
 [412000.]
 [480000.]
 [417000.]
 [585000.]
 [483000.]
 [443725.]
 [450000.]
 [340000.]
 [585000.]]
Predicted value: 458572.5
 
query set: [ 0.01345102  0.01551285  0.01355605  0.01222503  0.00

...

distances: [0.01360344 0.06846957 0.01456711 ... 0.01173812 0.012533   0.01290823]
sorted indices: [2819 3645 1344 ... 1107 5226 2486]
selected indices: [2819 3645 1344 1792  910  802  223 1381 1971 1546]
selected outputs: [[1000000.]
 [ 800500.]
 [ 560000.]
 [ 678000.]
 [ 566000.]
 [ 249000.]
 [ 710000.]
 [ 725000.]
 [ 245000.]
 [ 686000.]]
Predicted value: 621950.0
 
query set: [ 0.01345102  0.01163464  0.01506227  0.01097271  0.00092298  0.
  0.          0.01550947  0.01216718  0.01265775  0.          0.01362588
  0.          0.01345121 -0.0134712   0.00797265  0.00233234]
distances: [0.01158034 0.06767943 0.01750773 ... 0.0088302  0.01221737 0.00572522]
sorted indices: [2382 3819 4475 ... 1107 5226 2486]
selected indices: [2382 3819 4475 3558  707 1009 2327 3176 4947  220]
selected outputs: [[270000.]
 [270000.]
 [316000.]
 [379000.]
 [275000.]
 [282500.]
 [314000.]
 [260000.]
 [289000.]
 [408200.]]
Predicted value: 306370.0
 
query set: [ 0.01345102  0.01551285  0.01506227  0.0159

sorted indices: [ 944 3566 3126 ... 1107 5226 2486]
selected indices: [ 944 3566 3126 3954 2506  420 1642 2580 1943 1863]
selected outputs: [[312500.]
 [681000.]
 [359950.]
 [320900.]
 [490000.]
 [549500.]
 [595000.]
 [800000.]
 [299900.]
 [263000.]]
Predicted value: 467175.0
 
query set: [ 0.01345102  0.01163464  0.01506227  0.00918368  0.00042569  0.
  0.          0.0116321   0.01390535  0.0095621   0.00383765  0.01361223
  0.          0.01347489 -0.01346911  0.01080166  0.00064576]
distances: [0.01075221 0.06742242 0.01491098 ... 0.0094535  0.01273039 0.00407471]
sorted indices: [ 945 5072 5496 ... 1107 5226 2486]
selected indices: [ 945 5072 5496 4740 1590 5346 5233 5033 5365 4333]
selected outputs: [[416000.]
 [436110.]
 [453000.]
 [413450.]
 [670000.]
 [526000.]
 [415000.]
 [425000.]
 [379500.]
 [389999.]]
Predicted value: 452405.9
 
query set: [ 0.01345102  0.01551285  0.01204982  0.01300028  0.00282512  0.
  0.          0.01938684  0.01216718  0.00804868  0.02584019  0.01348942

selected indices: [2065 4091  573 4006 3652 3003 3815 1612 4063 4830]
selected outputs: [[336500.]
 [507000.]
 [529219.]
 [635000.]
 [302000.]
 [402000.]
 [265000.]
 [317500.]
 [509000.]
 [329000.]]
Predicted value: 413221.9
 
query set: [ 0.01345102  0.01163464  0.01054359  0.00727539  0.00239076  0.
  0.          0.01550947  0.01216718  0.00839264  0.          0.01340072
  0.          0.01348702 -0.01345117  0.00861561  0.0029205 ]
distances: [0.00602657 0.06829944 0.0122419  ... 0.0129002  0.01614843 0.00758064]
sorted indices: [4100 1301  505 ... 1107 5226 2486]
selected indices: [4100 1301  505 4007 4076 1223 4804   39 4625 1771]
selected outputs: [[239950.]
 [255000.]
 [347000.]
 [455000.]
 [315450.]
 [195000.]
 [208000.]
 [212000.]
 [295000.]
 [246000.]]
Predicted value: 276840.0
 
query set: [ 0.01345102  0.01163464  0.00602491  0.00572489  0.00274679  0.
  0.          0.0116321   0.01216718  0.00660404  0.          0.01344848
  0.          0.01349717 -0.01345216  0.00848702  0

sorted indices: [5174 4535 2776 ... 1107 5226 2486]
selected indices: [5174 4535 2776   19 5047 4970  799 5209 1583 5477]
selected outputs: [[577000.]
 [539900.]
 [729000.]
 [301000.]
 [400000.]
 [523000.]
 [325000.]
 [437000.]
 [574000.]
 [255000.]]
Predicted value: 466090.0
 
query set: [ 0.01345102  0.01551285  0.0165685   0.02117018  0.00210536  0.
  0.          0.0116321   0.01564352  0.0244212   0.          0.01373505
  0.          0.01349239 -0.0134406   0.02018882  0.0022784 ]
distances: [0.02716689 0.06911127 0.02913575 ... 0.01369445 0.01030306 0.02171423]
sorted indices: [5104 5248 5032 ... 1107 5226 2486]
selected indices: [5104 5248 5032 5386 5160 1585 2006 5252 3243 2489]
selected outputs: [[859990.]
 [799000.]
 [650000.]
 [780000.]
 [775000.]
 [846450.]
 [616000.]
 [850000.]
 [320000.]
 [680000.]]
Predicted value: 717644.0
 
query set: [ 0.01345102  0.00775643  0.01807473  0.01317918  0.0012575   0.
  0.          0.0116321   0.01738169  0.01031882  0.01816489  0.01370094

distances: [0.01548006 0.06758348 0.01877709 ... 0.00434666 0.00437482 0.00968326]
sorted indices: [5486 2542 3290 ... 1107 5226 2486]
selected indices: [5486 2542 3290 5110 4470 3229 3374 3483 3261 1729]
selected outputs: [[337500.]
 [597400.]
 [291375.]
 [320000.]
 [419950.]
 [425000.]
 [280000.]
 [339995.]
 [430000.]
 [332500.]]
Predicted value: 377372.0
 
query set: [ 0.01345102  0.01551285  0.01506227  0.01956602  0.00171883  0.
  0.          0.0116321   0.01564352  0.02257069  0.          0.01374187
  0.          0.01338484 -0.01344589  0.016704    0.00195308]
distances: [0.0232462  0.06833606 0.02639181 ... 0.01036245 0.00696718 0.01797198]
sorted indices: [3080 5181 3336 ... 1107 5226 2486]
selected indices: [3080 5181 3336 5364 1993 2799 5099 5125  394 3396]
selected outputs: [[725000.]
 [498445.]
 [419000.]
 [757500.]
 [580135.]
 [400000.]
 [820000.]
 [648000.]
 [810000.]
 [675000.]]
Predicted value: 633308.0
 
query set: [ 0.01345102  0.01163464  0.01506227  0.01329845  0.00